# Explorando dados das Boias Piratas


http://www.goosbrasil.org/pirata/dados

In [ ]:
import pandas as pd


url = "https://raw.githubusercontent.com/ocefpaf/2018-Jun-SWC-Floripa/master/data/dados_pirata.csv"

df = pd.read_csv(
    url,
    index_col="datahora",
    parse_dates=True,
    usecols=range(1, 15),
)

df.head()

## Exercício: check a documentação do pandas, procure por *na_values*. O que isso faz? Como podemos usar aqui?

In [ ]:
df = pd.read_csv(
    url,
    index_col="datahora",
    parse_dates=True,
    usecols=range(1, 15),
    na_values=-99999,
)

df.head()

Truque para remover o *t_* e ordenar as colunas por profundidade.

In [ ]:
df.columns = [
    f"{col.split('_')[1]:0>3}"
    for col in df.columns
]

df.sort_index(axis=1, inplace=True)
df.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(13, 5.25))
df.plot(ax=ax);

## Exercício: existe um "gap" de dados faltando bem grande entre 2014-2016, cheque a documentação do método *.dropna()* e veja como podemos utilizá-lo aqui para "limpar" os dados um pouco.

In [ ]:
df.dropna(axis=1, how="all", inplace=True)

df.head()

In [ ]:
desc = df.describe()

desc

## Exercício: usando o dataframe *desc* calcule a variância dos dados em cada profundidade.

In [ ]:
desc.loc["std"] ** 2

O que está acontecendo em 180 m?

In [ ]:
grouped = df.groupby(pd.Grouper(freq="Y"))

year = grouped.mean()
year = year.T
year

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))

ax.plot(year.values, year.index, "o-")
ax.invert_yaxis();

## Exercício:

- repita o `groupby` mas use meses ao invés de anos;
- plot as series temporais das médias dos meses.

In [ ]:
grouped = df.groupby(pd.Grouper(freq="M"))

monthly = grouped.mean()

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))

ax.plot(monthly.T.values, monthly.T.index, "o-")
ax.invert_yaxis();

In [ ]:
monthly.plot();

Uma forma de comparar as séries temporais com variâncias diferentes é normalizar pelo z-score:

$$ z = \frac{x - \mu}{\sigma} $$

In [ ]:
zscore = (monthly - monthly.mean(axis=0)) / monthly.std()

zscore.head()

In [ ]:
import hvplot.pandas


zscore.hvplot()

# Decompor o sinal

$$ y = trend + seasonal + residual $$

In [ ]:
signal = df["001"].copy()

mean = signal.mean()
mask = signal.isnull()

## Tendência

In [ ]:
import numpy as np
from scipy.signal import detrend


s = (signal-mean).fillna(0)
detrended_signal = detrend(s)

In [ ]:
detrended_signal[mask] = np.NaN
detrended_signal = pd.Series(detrended_signal, index=signal.index)

trend =  signal - detrended_signal

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)

signal.plot(ax=ax0, label="signal")
ax0.plot(signal.index, trend, label="trend")

detrended_signal.plot(ax=ax1, label="detrend");

Quais anos tem dados faltando?

In [ ]:
set(detrended_signal[detrended_signal.isnull()].index.year)

Antes de interpolar um pouco disso vamos olhar as "estatisticas" dos dados para não "inventar muito."

In [ ]:
detrended_signal.describe()

In [ ]:
interped_signal = detrended_signal.interpolate(method="time")

interped_signal.plot(marker=".")
set(interped_signal[interped_signal.isnull()].index.year)

In [ ]:
interped_signal.describe()

Exercício: esse tipo de interolação é aceitável? Olhe a documentação do `.interpolate` e procure uma opção que nos ajude a melhorar isso!

In [ ]:
interped_signal = detrended_signal.interpolate(
    method="time",
    limit=1,
)


interped_signal.plot(marker=".")
interped_signal.describe()

In [ ]:
set(interped_signal[interped_signal.isnull()].index.year)

In [ ]:
interped_signal_cutted = interped_signal.loc["2005":"2010-12"]

## Sazonalidade + ruído

$$ a\cos{(2\pi x + b)} $$

In [ ]:
def cosine(x, a, b):
    return a * np.cos(2 * np.pi * x + b)

In [ ]:
t = interped_signal_cutted.index.values
t = t - t[0]
t = t / np.timedelta64(1, "D") / 365

In [ ]:
from scipy.optimize import curve_fit
from matplotlib.dates import date2num


(a, b), pcov = curve_fit(
    cosine,
    t,
    interped_signal_cutted,
)

In [ ]:
a, b

In [ ]:
fitted = cosine(t, a, b)
res = interped_signal_cutted - fitted

fig, (ax0, ax1) = plt.subplots(
    nrows=2,
    sharex=True,
    figsize=(9, 5.75)
)

interped_signal_cutted.plot(ax=ax0, label="signal")
ax0.plot(
    interped_signal_cutted.index,
    fitted,
    label="ajuste"
)

ax0.legend()

res.plot(ax=ax1);

# Exercício: escolha outra boia e reproduza as análises desse notebook.


# Desafio 0: refaça o ajuste sem tirar a média/tendência e modificando a equação para achar *c*, ou seja, a tendência.

$$ a\cos{(2\pi x + b)} + c $$

# Desafio 1: parece existir um sinal mais longo no resíduo. Faça média vertical da camada de mistura (0-60), e refaça a análise com a série dessa média vertical. Isso deixara o sinal mais "robusto" (ou não ;-p).